<a href="https://colab.research.google.com/github/evroth/Congressional_Analytics/blob/main/Congressional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hypothesis:
Blue Districts will have more starbucks per person than Red Districts. 


## About:

This document contais the content for our first project in the Cloud Computing Class. The analysis below goes attempts to look at the number of starbucks locations per person in each congressional district, then look at whether the district is red or blue.

We will gather data for Starbucks locations off of Kaggle (Feb 2017, https://www.kaggle.com/datasets/starbucks/store-locations?resource=download).

Census data from Census.gov for Total Population, District Number, and State (https://www.census.gov/data/developers/data-sets/acs-1year.html).

Lastly data from some source on Political Affiliation, State, and District Number(http://goodcsv.com/politics/us-house-of-representatives-2020/). This could be an issue because it is the 2020 reps, not 2017 which we should look for.

We will do some EDA and merge the datasets together to perform our analysis.
The initial shape file I found is from https://cdmaps.polisci.ucla.edu/.


The first step here will be to get our data on Starbucks locations, note that you will need to download this file from Kaggle to the files colab can access for this to work.

In [17]:
# starbucks locations
## I am not sure how to always have this data downloaded, might need to re-upload it to colab every time?
import pandas as pd
from geopy.geocoders import Nominatim

# read store location data into a dataframe
store_df = pd.read_csv("starbucks_locations.csv")

# select only US stores
store_df = store_df[store_df['Country'] == "US"]

# Select the "Store Number", "State/Province", "Longitude", and "Latitude" columns
store_df = store_df[["Store Number", "State/Province", "Longitude", "Latitude"]]

# print the dataframe
print(store_df)

       Store Number State/Province  Longitude  Latitude
11964   3513-125945             AK    -149.78     61.21
11965   74352-84449             AK    -149.84     61.14
11966  12449-152385             AK    -149.85     61.11
11967  24936-233524             AK    -149.89     61.13
11968    8973-85630             AK    -149.86     61.14
...             ...            ...        ...       ...
25567   74385-87621             WY    -105.59     41.32
25568   73320-24375             WY    -105.56     41.31
25569  22425-219024             WY    -105.56     41.31
25570  10849-103163             WY    -109.25     41.58
25571  10769-102454             WY    -106.94     44.77

[13608 rows x 4 columns]


These next two chunks install libraries and download the zipfile for district shapes.

In [18]:
# Install Library
%%capture
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas
import geopandas as gpd

In [13]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

zipurl = 'https://cdmaps.polisci.ucla.edu/shp/districts114.zip'

with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall()

Here we use geopandas to find what Congressional District each store is located in.

In [21]:
# Convert Lat and Long to Congressional Districts
import geopandas as gpd

# convert the store location data into a geodataframe
store_gdf = gpd.GeoDataFrame(store_df, geometry=gpd.points_from_xy(store_df.Longitude, store_df.Latitude))

# read congressional district data into a geodataframe
district_gdf = gpd.read_file("/content/districtShapes/districts114.shp")

# merge the store location data with the congressional district data
merged_gdf = gpd.sjoin(store_gdf, district_gdf, op='within')

# select only the columns we need
merged_gdf = merged_gdf[["Store Number", "State/Province", "STATENAME", "DISTRICT"]]

# print the merged data
merged_gdf.head(100)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3249: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-21-9ea47f299da6>:11: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4269

  merged_gdf = gpd.sjoin(store_gdf, district_gdf, op='within')


,Store Number,State/Province,STATENAME,DISTRICT
11964,3513-125945,AK,Alaska,0
11965,74352-84449,AK,Alaska,0
11966,12449-152385,AK,Alaska,0
11967,24936-233524,AK,Alaska,0
11968,8973-85630,AK,Alaska,0
...,...,...,...,...
12086,76606-101463,AL,Alabama,3
12087,9738-97528,AL,Alabama,3
12023,24427-237908,AL,Alabama,7
12024,18675-189964,AL,Alabama,7


The next step is to read in the ACS 2017 data and format it into a usuable table.